In [15]:
import tabula
import re
import glob
import os 
from PIL import Image 
import pytesseract
import sys 
from pdf2image import convert_from_path 
from PyPDF2 import PdfFileReader
import pandas as pd

pdf_list = [f for f in glob.glob( '*.pdf')]
print(pdf_list)

['M34 - 2016-05.pdf', 'M34 - 2016-02.pdf', 'M34 - 2016-04.pdf']


In [16]:
def handle_first_rows(df):
    found = False
    start = 0
    end = 0
    #print(df.iloc[0:5])
    for (x,y) in df.iterrows():
        for t in y :
            if type(t) == str :
                if len(re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}", t)) != 0  :
                    end = x
                    #print(start,end)
                    initial = df.iloc[end]['Descrizione attività']
                    data = ''
                    for i in range(start,end):
                        #print(df.iloc[i]['Descrizione attività'])
                        #print(y['Descrizione attività'])
                        #print(df.iloc[end])
                        #df.iloc[end].set_value('Descrizione attività', df.iloc[i]['Descrizione attività'] + initial)
                        #print(y['Descrizione attività'] +' '+ str(df.iloc[i]['Descrizione attività']))
                        data +=  str(df.iloc[i]['Descrizione attività'])
                    print(data)
                    dfc = df.copy()
                    df.loc[:end]['Descrizione attività'] = data+' '+ y['Descrizione attività']  
                    found = True
        if found ==True:
            break
    #print(df.iloc[end]['Descrizione attività'])
    print('this isthe first date found ',end)
    return end


def handle_main_rows(df,start,end,pos):
    for (x,y) in df.iterrows():
        if x >=pos:
            for t in y :
                if type(t) == str :
                    if len(re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}", t)) != 0  :
                        #print(re.findall(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}", t),x)
                        if start == end:
                            end = x
                            #print(start,end)
                            data = ''
                            for i in range(start+1,end):
                            #print(i)
                                data +=  str(df.iloc[i]['Descrizione attività'])
                        #print(data)
                        #print(df.iloc[start]['Descrizione attività']+' '+data )
                            data_to_save = df.iloc[start]['Descrizione attività']+' '+data 
                            #print('data that will be written :',data_to_save)
                        
                            #print('this will be replaced',df.loc[start]['Descrizione attività'],'++++')
                            
                            df.loc[start,'Descrizione attività']= data_to_save
                        else :
                            start = end
                            end = x
                            #print("strat:",start,"end:",end)
                            data = ''
                            for i in range(start+1,end):
                            #print(i)
                                data +=  str(df.iloc[i]['Descrizione attività'])
                        #print(data)
                        #print(df.iloc[start]['Descrizione attività']+' '+data )
                            data_to_save = df.iloc[start]['Descrizione attività']+' '+data 
                            #print('data that will be written :',data_to_save)
                        
                            #print('this will be replaced',df.loc[start]['Descrizione attività'],'++++')
                            
                            df.loc[start,'Descrizione attività']= data_to_save
            last_date = end
    #print(df)
    return end


            

def handle_last_rows(df,last_date):
    data = " "
    for x,y in df.iloc[last_date:len(df)-1].iterrows():
        if x > last_date :
            data += str(df.iloc[x]['Descrizione attività'])
    #print(df.loc[last_date,'Descrizione attività'] +" "+data)
    data = df.loc[last_date,'Descrizione attività'] +" "+data
    df.loc[last_date,'Descrizione attività'] = data

In [22]:
def extract_risora_umana(pdf_name):
    
    pages = convert_from_path(pdf_name, 500) # Store all the pages of the PDF in a variable  
    image_counter = 1   # Counter to store images of each page of PDF to image
    name = pdf_name.split('.')[0]
    outfile = str(name)+".txt" # prepare text file name 
    f = open(outfile, "a") # open the file to write into it 

    for page in pages: # Iterate through all the pages stored above 
      # For each page, filename will be:  
    # PDF page 1 -> page_1.jpg 
    # PDF page 2 -> page_2.jpg 
    # PDF page 3 -> page_3.jpg 
    # .... 
    # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".jpg" # Declaring name for each page of PDF as JPG
      
    # Save the image of the page in system 
        page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
        image_counter = image_counter + 1
    # convert the image into text 
        text = str(((pytesseract.image_to_string(Image.open(filename)))))
        os.remove(filename)
    # write the text into a file in the HDD
        f.write(text) 
    f.close()
    risora_umana = " "
    with open(outfile,'rb') as f :
        lines = f.readlines()
        found = False
        for line in lines :
            #print(line.decode('utf-8'))
            if 'Risorsa Umana' in line.decode('utf-8') :
                #risora_umana = str(line)
                print(line.decode('utf-8'))
                risora_umana = line.decode('utf-8')
                break
        
        print(risora_umana[risora_umana.find(':')+1:])
    os.remove(outfile)
    return risora_umana[risora_umana.find(':')+1:]


all_data = pd.DataFrame()
for pdf in sorted(pdf_list) :
    risora_umana = extract_risora_umana(pdf)
    start = 0
    end = 0
    pos = 1
    last_date = 0
    pdf_name = pdf.split('.')[0]
    tables = tabula.read_pdf(pdf, pages = "all", multiple_tables = True)
    dfc = tables[0].copy()
    if len(tables) != 0 :
        df = tables[0]
        df['Risora Umana'] = risora_umana
        pos = handle_first_rows(df)
        last_date = handle_main_rows(df,start,end,pos)
        handle_last_rows(df,last_date)
        last_row = df.iloc[len(df)-1]
        df = df.dropna()
        #df = df.append([last_row],ignore_index=True)
        df.to_csv("{0}.csv".format(pdf_name), sep='\t', encoding='utf-8')
        all_data = all_data.append(df,ignore_index=True)

all_data.to_csv("all_data.csv", sep='\t', encoding='utf-8')
        

Risorsa Umana: Gagliardelli Ugo

 Gagliardelli Ugo

Attività di macro classificazione delle possibili condizioni di
this isthe first date found  1


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Risorsa Umana: Gagliardelli Ugo

 Gagliardelli Ugo


this isthe first date found  0


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Risorsa Umana: Gagliardelli Ugo

 Gagliardelli Ugo


this isthe first date found  0


/usr/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
